In [1]:
# ==========================================
# UJI COBA MANUAL: LIGHTGBM (TEMAN)
# ==========================================
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import string
import nltk
from nltk.corpus import stopwords
import torch
import os

# --- 1. KONFIGURASI ---
if 'ROOT_DIR' not in locals():
    ROOT_DIR = os.getcwd()
    if not os.path.exists(os.path.join(ROOT_DIR, 'models')):
        ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
DATA_PATH = os.path.join(ROOT_DIR, 'data', 'processed', 'dataset_training_FULL_COMPLETE.csv')
EMB_FILE = os.path.join(MODEL_DIR, 'corpus_embeddings.pt')

# Path ke model teman Anda (Pastikan file ini ada di folder models)
LGBM_PATH = os.path.join(MODEL_DIR, 'lightgbm (2) (1).pkl')

# URUTAN FITUR WAJIB (Sesuai isi perut file .pkl teman Anda)
LGBM_FEATURES = ['sbert_sim', 'overlap_score', 'jaccard_score', 'bm25_score']

print("⚙️ MEMUAT ENGINE LIGHTGBM...")

# --- 2. LOAD DATA & INDEX (SAMA SEPERTI SEBELUMNYA) ---
# Kita tetap pakai data dan embedding yang sama, cuma ganti "Juri"-nya saja.

# A. Load Data Teks & Metadata
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError("Dataset FULL_COMPLETE tidak ditemukan!")

df_full = pd.read_csv(DATA_PATH)
df_full.columns = df_full.columns.str.strip().str.lower()
df_index = df_full.drop_duplicates(subset=['text']).copy()
unique_tafsirs = df_index['text'].astype(str).tolist()

metadata_map = {}
for _, row in df_index.iterrows():
    key = str(row['text']).strip()
    metadata_map[key] = {
        'lokasi': row.get('ayat_asal', '?'),
        'arabic': row.get('arabic', ''),
        'trans': row.get('translation', '')
    }

print(f"   -> Data Training: {len(unique_tafsirs)} dokumen.")

# B. Load Embedding Lama (.pt)
if not os.path.exists(EMB_FILE):
    raise FileNotFoundError("File corpus_embeddings.pt tidak ditemukan!")
corpus_embeddings = torch.load(EMB_FILE, map_location='cpu')
print("   -> Embedding Lama: Teruat.")

# C. Load Model LightGBM (Pickle)
print(f"   -> Memuat Model LightGBM: {os.path.basename(LGBM_PATH)}")
if not os.path.exists(LGBM_PATH):
    raise FileNotFoundError(f"File {LGBM_PATH} tidak ditemukan! Taruh di folder models dulu.")

try:
    # Coba load pakai pickle biasa (karena format .pkl)
    with open(LGBM_PATH, 'rb') as f:
        lgbm_model = pickle.load(f)
    print("      ✅ Berhasil load via Pickle.")
except:
    # Fallback joblib
    lgbm_model = joblib.load(LGBM_PATH)
    print("      ✅ Berhasil load via Joblib.")

# D. Setup SBERT & BM25
sbert_path = os.path.join(MODEL_DIR, 'sbert_finetuned_quran')
try:
    sbert_model = SentenceTransformer(sbert_path, device='cpu', trust_remote_code=True)
except:
    sbert_model = SentenceTransformer(sbert_path, device='cpu')

try: nltk.download('stopwords', quiet=True); stop_words = set(stopwords.words('indonesian'))
except: stop_words = set(['yang', 'dan', 'di'])

def clean_tokens(text):
    text = str(text).lower().translate(str.maketrans('', '', string.punctuation))
    return [w for w in text.split() if w not in stop_words]

corpus_tokens = [clean_tokens(t) for t in unique_tafsirs]
bm25 = BM25Okapi(corpus_tokens)

print("\n✅ SIAP TESTING LIGHTGBM!")

# --- 3. FUNGSI PENCARIAN (KHUSUS LIGHTGBM) ---
def cari_pake_lgbm(query_text, threshold=0.1): # LGBM kadang butuh threshold lebih rendah/tinggi tergantung training
    print(f"\n🔍 QUERY: '{query_text}'")
    print("=" * 60)
    
    # 1. Encode Query
    query_vec = sbert_model.encode(query_text, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, corpus_embeddings, top_k=50)[0]
    
    candidates = []
    q_toks = clean_tokens(query_text)
    
    for hit in hits:
        idx = hit['corpus_id']
        txt = unique_tafsirs[idx]
        
        # Hitung Fitur
        t_toks = corpus_tokens[idx]
        sq, st = set(q_toks), set(t_toks)
        ov = len(sq & st) / len(sq) if sq else 0
        jac = len(sq & st) / (len(sq | st) + 1e-9)
        bm25_s = bm25.get_batch_scores(q_toks, [idx])[0]
        
        candidates.append({
            'text_key': txt,
            'sbert_sim': hit['score'],
            'bm25_score': bm25_s,
            'overlap_score': ov,
            'jaccard_score': jac
        })
    
    # 2. Re-Ranking pakai LightGBM
    df_cand = pd.DataFrame(candidates)
    
    # SUSUN KOLOM SESUAI URUTAN MODEL TEMAN ANDA (WAJIB!)
    X_pred = df_cand[LGBM_FEATURES]
    
    # Prediksi
    try:
        # Jika model klasifikasi (keluar probabilitas 0-1)
        scores = lgbm_model.predict_proba(X_pred)[:, 1]
    except:
        # Jika model regresi (keluar skor mentah)
        scores = lgbm_model.predict(X_pred)
        
    df_cand['final_score'] = scores
    
    # 3. Tampilkan
    results = df_cand[df_cand['final_score'] > threshold].sort_values('final_score', ascending=False)
    
    if len(results) == 0:
        print("❌ Tidak ada hasil (Coba turunkan threshold).")
    else:
        print(f"✅ Top 3 Hasil (Engine: LightGBM):\n")
        for i, row in enumerate(results.head(3).iterrows()):
            _, data = row
            key = str(data['text_key']).strip()
            score = data['final_score']
            info = metadata_map.get(key)
            
            print(f"🏅 [RANK {i+1}] Skor: {score:.4f}")
            if info:
                print(f"📍 {info['lokasi']}")
                print(f"🇸🇦 {info['arabic'][:50]}...")
                print(f"🇮🇩 {info['trans'][:100]}...")
            print(f"💬 {key[:100]}...")
            print("-" * 50)

# --- 4. TEST ---
cari_pake_lgbm("Kisah sapi betina bani israil")
cari_pake_lgbm("Hukum warisan bagi perempuan")

⚙️ MEMUAT ENGINE LIGHTGBM...


FileNotFoundError: Dataset FULL_COMPLETE tidak ditemukan!

In [4]:
# --- KONFIGURASI PATH ---
# Notebook ada di folder scripts/, jadi naik 1 level ke root
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'processed')

# FILE-FILE YANG DIBUTUHKAN
CSV_PATH = os.path.join(DATA_DIR, 'dataset_training_FULL_COMPLETE.csv')
EMB_FILE = os.path.join(MODEL_DIR, 'corpus_embeddings.pt')
LGBM_PATH = os.path.join(MODEL_DIR, 'lightgbm (2) (1).pkl') # Model Teman Anda

# URUTAN FITUR WAJIB (Sesuai model teman Anda)
LGBM_FEATURES = ['sbert_sim', 'overlap_score', 'jaccard_score', 'bm25_score']

print("✅ Path berhasil diset.")
print(f"📂 Root Dir: {ROOT_DIR}")
print(f"📂 Model Dir: {MODEL_DIR}")
print(f"📂 Data Dir: {DATA_DIR}")
print(f"\n🔍 Checking files:")
print(f"   CSV exists: {os.path.exists(CSV_PATH)}")
print(f"   Embedding exists: {os.path.exists(EMB_FILE)}")
print(f"   LightGBM exists: {os.path.exists(LGBM_PATH)}")

✅ Path berhasil diset.
📂 Root Dir: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir
📂 Model Dir: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\models
📂 Data Dir: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\data\processed

🔍 Checking files:
   CSV exists: True
   Embedding exists: True
   LightGBM exists: True


In [5]:
print("⚙️ Memuat Dataset Lengkap...")

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File {CSV_PATH} tidak ditemukan!")

# 1. Baca CSV
df_full = pd.read_csv(CSV_PATH)
df_full.columns = df_full.columns.str.strip().str.lower()

# 2. Ambil Tafsir Unik (Untuk Pencarian)
# Urutan ini harus SAMA PERSIS dengan urutan saat corpus_embeddings.pt dibuat
df_index = df_full.drop_duplicates(subset=['text']).copy()
unique_tafsirs = df_index['text'].astype(str).tolist()

# 3. Buat Kamus Metadata (Untuk Tampilan Arab/Terjemah)
metadata_map = {}
for _, row in df_index.iterrows():
    key = str(row['text']).strip()
    metadata_map[key] = {
        'lokasi': row.get('ayat_asal', '?'),
        'arabic': row.get('arabic', ''),
        'trans': row.get('translation', '')
    }

print(f"✅ Data Termuat: {len(unique_tafsirs)} dokumen tafsir unik.")
print("✅ Metadata Map Siap.")

⚙️ Memuat Dataset Lengkap...
✅ Data Termuat: 6031 dokumen tafsir unik.
✅ Metadata Map Siap.


In [6]:
print("⚙️ Memuat Embedding & SBERT...")

# 1. Load Embedding Lama (.pt)
if os.path.exists(EMB_FILE):
    corpus_embeddings = torch.load(EMB_FILE, map_location=torch.device('cpu'))
    print(f"✅ Embedding Lama Berhasil Diload: {len(corpus_embeddings)} vektor.")
else:
    raise FileNotFoundError("File corpus_embeddings.pt hilang! Anda harus punya file ini.")

# 2. Load SBERT (Hanya untuk encode query user nanti)
SBERT_FOLDER = os.path.join(MODEL_DIR, 'sbert_finetuned_quran')
try:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu', trust_remote_code=True)
    print("✅ SBERT Finetuned Loaded.")
except:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')
    print("✅ SBERT (Default) Loaded.")

⚙️ Memuat Embedding & SBERT...
✅ Embedding Lama Berhasil Diload: 6031 vektor.


The tokenizer you are loading from 'c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\models\sbert_finetuned_quran' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ SBERT Finetuned Loaded.


In [7]:
print("⚙️ Memuat Model LightGBM...")

if not os.path.exists(LGBM_PATH):
    raise FileNotFoundError(f"Model {LGBM_PATH} tidak ditemukan.")

try:
    # Cara 1: Pakai Pickle biasa
    with open(LGBM_PATH, 'rb') as f:
        lgbm_model = pickle.load(f)
    print("✅ Model dimuat via Pickle.")
except:
    try:
        # Cara 2: Pakai Joblib
        lgbm_model = joblib.load(LGBM_PATH)
        print("✅ Model dimuat via Joblib.")
    except Exception as e:
        print(f"❌ Gagal memuat model: {e}")

# Cek Fitur (Opsional, untuk memastikan urutan benar)
try:
    if hasattr(lgbm_model, 'feature_name_'):
        print(f"ℹ️ Fitur Model: {lgbm_model.feature_name_}")
    elif hasattr(lgbm_model, 'booster_'):
        print(f"ℹ️ Fitur Model: {lgbm_model.booster_.feature_name()}")
except:
    pass

print(f"⚠️ PASTIKAN URUTAN FITUR KITA ADALAH: {LGBM_FEATURES}")

⚙️ Memuat Model LightGBM...
✅ Model dimuat via Joblib.
ℹ️ Fitur Model: ['sbert_sim', 'overlap_score', 'jaccard_score', 'bm25_score']
⚠️ PASTIKAN URUTAN FITUR KITA ADALAH: ['sbert_sim', 'overlap_score', 'jaccard_score', 'bm25_score']


c:\Users\Farhan\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
print("⚙️ Menyiapkan BM25...")

# Setup Stopwords
try:
    nltk.download('stopwords', quiet=True)
    stop_words = set(stopwords.words('indonesian'))
except:
    stop_words = set(['yang', 'dan', 'di'])

def clean_tokens(text):
    text = str(text).lower().translate(str.maketrans('', '', string.punctuation))
    return [w for w in text.split() if w not in stop_words]

# Build Index
corpus_tokens = [clean_tokens(t) for t in unique_tafsirs]
bm25 = BM25Okapi(corpus_tokens)

print("✅ BM25 Siap.")

⚙️ Menyiapkan BM25...
✅ BM25 Siap.


In [9]:
def cari_lightgbm(query_text, threshold=0.1):
    print(f"\n🔍 QUERY: '{query_text}'")
    print("=" * 60)
    
    # 1. Retrieval (SBERT) - Cari 50 Kandidat Awal
    query_vec = sbert_model.encode(query_text, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, corpus_embeddings, top_k=50)[0]
    
    candidates = []
    q_toks = clean_tokens(query_text)
    
    # 2. Hitung Fitur (On-The-Fly)
    for hit in hits:
        idx = hit['corpus_id']
        txt = unique_tafsirs[idx]
        t_toks = corpus_tokens[idx]
        
        # Hitung skor-skor manual
        sq, st = set(q_toks), set(t_toks)
        ov = len(sq & st) / len(sq) if sq else 0
        jac = len(sq & st) / (len(sq | st) + 1e-9)
        bm25_s = bm25.get_batch_scores(q_toks, [idx])[0]
        
        candidates.append({
            'text_key': txt,
            'sbert_sim': hit['score'],
            'bm25_score': bm25_s,
            'overlap_score': ov,
            'jaccard_score': jac
        })
    
    # 3. Prediksi Pakai LightGBM
    df_cand = pd.DataFrame(candidates)
    
    # WAJIB: Susun kolom sesuai urutan model teman Anda
    X_pred = df_cand[LGBM_FEATURES]
    
    try:
        # Klasifikasi (Probabilitas)
        scores = lgbm_model.predict_proba(X_pred)[:, 1]
    except:
        # Regresi (Skor Langsung)
        scores = lgbm_model.predict(X_pred)
        
    df_cand['final_score'] = scores
    
    # 4. Filter & Tampilkan
    results = df_cand[df_cand['final_score'] > threshold].sort_values('final_score', ascending=False)
    
    if len(results) == 0:
        print("❌ Tidak ada hasil relevan.")
    else:
        print(f"✅ Top 3 Hasil (Engine: LightGBM):\n")
        for i, row in enumerate(results.head(3).iterrows()):
            _, data = row
            key = str(data['text_key']).strip()
            score = data['final_score']
            
            # Ambil Info Lengkap (Arab/Terjemah)
            info = metadata_map.get(key)
            
            print(f"🏅 [RANK {i+1}] Skor: {score:.4f}")
            if info:
                print(f"📍 {info['lokasi']}")
                print(f"🇸🇦 {info['arabic'][:60]}...") # Dipotong dikit biar rapi di print
                print(f"🇮🇩 {info['trans'][:100]}...")
            
            print(f"💬 TAFSIR: {key[:100]}...")
            print("-" * 50)

In [10]:
# Tes Query
cari_lightgbm("Kisah sapi betina bani israil")
cari_lightgbm("Hukum warisan bagi perempuan")


🔍 QUERY: 'Kisah sapi betina bani israil'
[LightGBM] [Warning] Unknown parameter: gamma
✅ Top 3 Hasil (Engine: LightGBM):

🏅 [RANK 1] Skor: 0.8973
📍 QS. Ṭāhā : Ayat 96
🇸🇦 قَالَ بَصُرْتُ بِمَا لَمْ يَبْصُرُوْا بِهٖ فَقَبَضْتُ قَبْضَ...
🇮🇩 Dia (Samiri) menjawab, “Aku melihat sesuatu yang tidak mereka lihat. Kemudian, aku ambil segenggam (...
💬 TAFSIR: Mendapat pertanyaan dari Nabi Musa, dia menjawab, “Aku melihat dan mengetahui sesuatu yang tidak mer...
--------------------------------------------------
🏅 [RANK 2] Skor: 0.8831
📍 QS. Al-A‘rāf : Ayat 105
🇸🇦 حَقِيْقٌ عَلٰٓى اَنْ لَّآ اَقُوْلَ عَلَى اللّٰهِ اِلَّا الْ...
🇮🇩 Wajib atasku tidak mengatakan (sesuatu) terhadap Allah, kecuali yang hak (benar). Sungguh, aku datan...
💬 TAFSIR: Sebagai seorang nabi dan rasul yang bertugas menyampaikan pesan Allah, aku wajib mengatakan yang seb...
--------------------------------------------------
🏅 [RANK 3] Skor: 0.8810
📍 QS. Al-Baqarah  : Ayat 51
🇸🇦 وَاِذْ وٰعَدْنَا مُوْسٰىٓ اَرْبَعِيْنَ لَيْلَةً ث